# Column 3 as target column:
# Team Members:
# USN: 01FB15ECS278 Name: Shashank Saran
# USN: 01FB15ECS286 Name: Shreyas V Patil
# USN: 01FB15ECS289 Name: Siddhanth Vinay
# USN: 01FB15ECS290 Name: Siddharth Ganesan


In [211]:
import tensorflow as tf
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [212]:
colnames=['x1', 'x2', 'y1', 'y2'] 
dataset_df1 = pd.read_csv('algebra.csv',names=colnames,header=None)

In [213]:
feature1 = tf.feature_column.numeric_column('x1', dtype=tf.float32, shape=())
feature2 = tf.feature_column.numeric_column('x2', dtype=tf.float32, shape=())
label1 = tf.feature_column.numeric_column('y1', dtype=tf.float32, shape=())
label2 = tf.feature_column.numeric_column('y2', dtype=tf.float32, shape=())
feature_cols = [feature1,feature2]
feature_name= ['x1','x2']
label_name = 'y1'
feature_ndarray1 = dataset_df1[feature_name].astype(np.float32)
label_ndarray1 = dataset_df1[label_name].astype(np.float32)
x_train1, x_test1, y_train1, y_test1 = train_test_split(feature_ndarray1, label_ndarray1, random_state=0, test_size=0.15)

In [214]:
def train_input1():
    _dataset = tf.data.Dataset.from_tensor_slices(({'x1': x_train1['x1'],'x2': x_train1['x2']}, y_train1))
    dataset = _dataset.batch(8)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

In [215]:
def val_input1():
    _dataset = tf.data.Dataset.from_tensor_slices(({'x1': x_test1['x1'],'x2': x_test1['x2']}, y_test1))
    dataset = _dataset.batch(32)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

In [216]:
def my_dnn_regression_fn(features, labels, mode, params):
    top = tf.feature_column.input_layer(features, params["feature_columns"])
    ctr=0
    for units in params.get("hidden_units"):
        nam="hidden_"+str(ctr)
        ctr+=1
        top = tf.layers.dense(inputs=top, units=units, activation=tf.nn.relu,name=nam)
    output_layer = tf.layers.dense(inputs=top, units=1,name="output")
    predictions = tf.squeeze(output_layer, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions={"Predict": predictions})
    average_loss = tf.losses.mean_squared_error(labels, predictions)
    batch_size = tf.shape(labels)[0]
    total_loss = tf.to_float(batch_size) * tf.to_float(average_loss) #tf.to_float
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = params.get("optimizer", tf.train.AdamOptimizer)
        optimizer = optimizer(params.get("learning_rate", None))
        train_op = optimizer.minimize(
        loss=average_loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
        mode=mode, loss=total_loss, train_op=train_op)
    assert mode == tf.estimator.ModeKeys.EVAL
    rmse = tf.metrics.root_mean_squared_error(labels, predictions)
    eval_metrics = {"rmse": rmse}
    return tf.estimator.EstimatorSpec(mode=mode,
    loss=total_loss,
    eval_metric_ops=eval_metrics)

In [217]:
class myDNNRegressorEstimator():
        def __init__(self,model):
            self.model = model
        def get_weights(self):
            layers = dict()
            list_layers = ["hidden_0","output"]
            for layer in list_layers:
                weights = self.model.get_variable_value(layer+"/kernel")
                weights=np.transpose(weights)
                bias = self.model.get_variable_value(layer+"/bias")
                layers[layer] = {"weights":weights,"bias":bias}
            return layers
        def predict(self,val1,val2):
                l1=[val1,val2]
                weights = self.model.get_variable_value("hidden_0/kernel")
                bias = self.model.get_variable_value("hidden_0/bias")
                weights=np.transpose(weights)
                vals=[]
                for i in weights:
                    vals.append(np.matmul(np.transpose(np.array(l1)),i))
                vals=np.add(vals,bias)
                out_weights=self.model.get_variable_value("output/kernel")
                out_bias=self.model.get_variable_value("output/bias")
                # The line below implements the ReLu activation function.
                vals[vals<0]=0
                vals=np.matmul(np.transpose(vals),out_weights)
                vals=np.add(vals,out_bias)
                return vals[0]

In [222]:
model = tf.estimator.Estimator(
      model_fn= my_dnn_regression_fn,
      params={
          "feature_columns": feature_cols,
          "learning_rate": 0.1,
          "optimizer": tf.train.AdamOptimizer,
          "hidden_units": [100000]
      })

model.train(input_fn=train_input1, steps=500)

In [223]:
eval_result = model.evaluate(input_fn=val_input1)
print(eval_result)

{'global_step': 500, 'loss': 10.957685, 'rmse': 0.58595294}


In [224]:
myEstimator = myDNNRegressorEstimator(model)
#weights = myEstimator.get_weights()
#print(weights)
list1=[[1,1],[1,2],[2,1],[3,1],[3,2],[5,3],[4,3],[4,2],[1,3]]
list2=[]
for i in list1:
    list2.append(myEstimator.predict(i[0],i[1]))
for i in range(len(list1)):
    print("X1: ",list1[i][0],"X2: ",list1[i][1],"\tValue: ",list2[i],"\tRounded Value: ",round(list2[i]))

X1:  1 X2:  1 	Value:  0.14925137481430184 	Rounded Value:  0.0
X1:  1 X2:  2 	Value:  -2.57243279926288 	Rounded Value:  -3.0
X1:  2 X2:  1 	Value:  2.6947587287310943 	Rounded Value:  3.0
X1:  3 X2:  1 	Value:  8.185203361086955 	Rounded Value:  8.0
X1:  3 X2:  2 	Value:  5.258861866655921 	Rounded Value:  5.0
X1:  5 X2:  3 	Value:  15.015205282085848 	Rounded Value:  15.0
X1:  4 X2:  3 	Value:  7.240164409484324 	Rounded Value:  7.0
X1:  4 X2:  2 	Value:  12.00453937160837 	Rounded Value:  12.0
X1:  1 X2:  3 	Value:  -7.786751607665861 	Rounded Value:  -8.0


# Hence, from the values of X1 and X2 and the rounded value, it is evident that the output value is the function X1^2 - X2^2